In [9]:
from model import BioCNN, MultiStreamBioCNN
import torch
from thop import profile
import pandas as pd

In [10]:
# instance models
model_bio = BioCNN()
model_ecg_ppg = BioCNN(in_channels=2)
model_multibio = MultiStreamBioCNN()

In [11]:
# random input for convolutional layers
random_input_bio = torch.rand(1, 1, 32, 32)
random_input_multibio = torch.rand(1, 2, 32, 32)

# random input for fully conected layers
random_input_bio2 = torch.rand(1, 128)
random_input_multibio2 = torch.rand(1, 256)

In [12]:
# calculate number flops and number total paramaters 
flops_bio, params_bio = profile(
    model_bio, inputs=(random_input_bio,))

flops_ecg_ppg, params_ecg_ppg = profile(
    model_ecg_ppg, inputs=(random_input_multibio,))

flops_multibio, params_multibio = profile(
    model_multibio, inputs=(random_input_multibio,))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch

In [13]:
# calculate number flops and number send paramaters 
_, param_bio_send = profile(
    model_bio.fc, inputs=(random_input_bio2,))

_, param_ecg_ppg_send = profile(
    model_ecg_ppg.fc, inputs=(random_input_bio2,))

_, param_multibio_send = profile(
    model_multibio.fc, inputs=(random_input_multibio2,))

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.


In [14]:
data = {
    'ECG': [flops_bio, params_bio],
    'PPG': [flops_bio, params_bio],
    'ECG_PPG': [flops_ecg_ppg, params_ecg_ppg],
    'Fusion': [flops_multibio, params_multibio],
    'ECG (Slipt)': [flops_bio, param_bio_send],
    'PPG (Slipt)': [flops_bio, param_bio_send],
    'ECG_PPG (Slipt)': [flops_ecg_ppg, param_ecg_ppg_send],
    'Fusion (Slipt)': [flops_multibio, param_multibio_send]
}

In [15]:
df = pd.DataFrame.from_dict(data, orient="index", columns=["FLOPs", "Send Parameters"])

# Convertendo para KB
df['Send Size (KB)'] = df['Send Parameters'] / 2**10
df.drop('Send Parameters', axis=1, inplace=True)

# Convertendo para mega FLOPS
df['Mega FLOPs'] = df['FLOPs']/1e6
df.drop('FLOPs', axis=1, inplace=True)

In [16]:
df.to_csv('flops and size.csv')

In [17]:
df

,Send Size (KB),Mega FLOPs
ECG,187.473633,1.409536
PPG,187.473633,1.409536
ECG_PPG,187.543945,1.483264
Fusion,347.895508,2.791936
ECG (Slipt),91.051758,1.409536
PPG (Slipt),91.051758,1.409536
ECG_PPG (Slipt),91.051758,1.483264
Fusion (Slipt),155.051758,2.791936
